In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re, json, glob, argparse
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode

import argparse 
import json # 
import glob
import logging
import os
import random

import timeit
from tqdm import tqdm

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler # Dataset을 보다 간편하게 사용하기 위한 함수
from torch.utils.data.distributed import DistributedSampler 
from fastprogress.fastprogress import master_bar, progress_bar # 학습 과정을 progress_bar로 예쁘게 표현하기 위해 사용하는 tool 
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
#from transformers import BertTokenizer, BertTokenizerFast, BertForQuestionAnswering
#from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

import gc

In [3]:
append=pd.read_csv('/content/drive/MyDrive/BIZ&AI_PAPER_11기/세희/memento/task/quesion&answering.csv')
append

,contexts,questions,answers,start_ids,end_ids
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,33,36
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,33,51
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,0,13
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,14,21
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,23,26
...,...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,14,21
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,23,28
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,35,43
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,44,52


In [4]:
append.columns

Index(['contexts', 'questions', 'answers', 'start_ids', 'end_ids'], dtype='object')

In [5]:
append=append.drop(['start_ids', 'end_ids'], axis=1)
append

,contexts,questions,answers
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집
...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다


In [6]:
append['date']=""
append

,contexts,questions,answers,date
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,
...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,


In [7]:
append['date'] = append['contexts'].str[0:13]
append

,contexts,questions,answers,date
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,2022년 06월 26일
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,2022년 06월 26일
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,2022년 06월 26일
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,2022년 06월 26일
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,2022년 06월 26일
...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,2020년 12월 14일
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,2020년 12월 14일
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,2020년 12월 14일
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,2020년 12월 14일


In [8]:
append.columns

Index(['contexts', 'questions', 'answers', 'date'], dtype='object')

In [9]:
append['date']=append['date'].str.replace('년', '-')

In [10]:
append

,contexts,questions,answers,date
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,2022- 06월 26일
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,2022- 06월 26일
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,2022- 06월 26일
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,2022- 06월 26일
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,2022- 06월 26일
...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,2020- 12월 14일
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,2020- 12월 14일
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,2020- 12월 14일
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,2020- 12월 14일


In [11]:
append['date']=append['date'].str.replace('월', '-')

In [12]:
append

,contexts,questions,answers,date
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,2022- 06- 26일
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,2022- 06- 26일
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,2022- 06- 26일
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,2022- 06- 26일
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,2022- 06- 26일
...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,2020- 12- 14일
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,2020- 12- 14일
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,2020- 12- 14일
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,2020- 12- 14일


In [13]:
append['date']=append['date'].str.replace('일', '')

In [14]:
append['date']=append['date'].str.replace(' ', '')

In [15]:
append=append[['date', 'contexts', 'questions', 'answers']]
append

,date,contexts,questions,answers
0,2022-06-26,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이
1,2022-06-26,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다
2,2022-06-26,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일
3,2022-06-26,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분
4,2022-06-26,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집
...,...,...,...,...
646,2020-12-14,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분
647,2020-12-14,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자
648,2020-12-14,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다
649,2020-12-14,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다


In [16]:
append['date']=pd.to_datetime(append['date'])

In [17]:
append['date']

0     2022-06-26
1     2022-06-26
2     2022-06-26
3     2022-06-26
4     2022-06-26
         ...    
646   2020-12-14
647   2020-12-14
648   2020-12-14
649   2020-12-14
650   2020-12-14
Name: date, Length: 651, dtype: datetime64[ns]

In [18]:
append=append.sort_values(by=['date'])

In [19]:
append

,date,contexts,questions,answers
103,2020-01-14,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 말했다 동기모임을 했다 ...,동기모임을 했다 오랜만에 친구를 만나서 좋았다를 말한 장소는 어디인가요?,싱싱오징어바다
105,2020-01-14,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 말했다 동기모임을 했다 ...,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 어떤 감정을 느꼈나요?,오랜만에 친구를 만나서 좋았다
106,2020-01-14,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 말했다 동기모임을 했다 ...,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 한 말은 무엇인가요?,동기모임을 했다 오랜만에 친구를 만나서 좋았다
107,2020-01-14,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 말했다 동기모임을 했다 ...,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 누구와 함께 행동했나요?,친구
102,2020-01-14,2020년 01월 14일 01시 14분에 싱싱오징어바다에서 말했다 동기모임을 했다 ...,동기모임을 했다 오랜만에 친구를 만나서 좋았다를 말한 시간은 몇 시 인가요?,01시 14분
...,...,...,...,...
33,2022-10-21,2022년 10월 21일 18시 20분에 설빙에서 말했다 딸기빙수를 먹었다 빙수 얼...,딸기빙수를 먹었다 빙수 얼음이 녹았다를 말한 시간은 몇 시 인가요?,18시 20분
34,2022-10-21,2022년 10월 21일 18시 20분에 설빙에서 말했다 딸기빙수를 먹었다 빙수 얼...,딸기빙수를 먹었다 빙수 얼음이 녹았다를 어떤 날짜에 말했나요?,2022년 10월 21일
35,2022-10-21,2022년 10월 21일 18시 20분에 설빙에서 말했다 딸기빙수를 먹었다 빙수 얼...,2022년 10월 21일 18시 20분에 설빙에서 무엇을 했나요?,딸기빙수를 먹었다
36,2022-10-21,2022년 10월 21일 18시 20분에 설빙에서 말했다 딸기빙수를 먹었다 빙수 얼...,딸기빙수를 먹었다 빙수 얼음이 녹았다를 말한 장소는 어디인가요?,설빙


In [ ]:
append.to_csv('quesion&answering_new.csv', index=False)